<a href="https://colab.research.google.com/github/dohyun93/hongong_mldl/blob/main/4_2_%ED%99%95%EB%A5%A0%EC%A0%81_%EA%B2%BD%EC%82%AC_%ED%95%98%EA%B0%95%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 확률적 경사하강법 (Stochastic Gradient Descent)

훈련세트 데이터가 계속해서 들어오는 상황이라고 생각해보자.

훈련 데이터가 점차 늘어남에따라 아래 방법으로 학습하는 것을 생각해볼 수 있다.

1) 매일 새로 들어오는 훈련 데이터를 포함해서 모델을 새로 학습한다.
-> 이 방법은 시간이 지날수록 훈련에 필요한 컴퓨팅 파워가 지속적으로 증가하여 좋지 않은 방법이다. 학습한 모델을 버리고 매일 더욱 증가하는 훈련 데이터로 학습을 해야한다.

2) 훈련데이터가 새로 들어온 만큼 앞서 수집한 훈련 데이터를 제거하고 매일 새로운 모델을 일정량의 훈련 데이터를 이용해 학습한다.
-> 이 방법은 앞서 중요한 훈련 데이터가 섞여있을 때 버려진다면 모델의 성능이 저하되는 문제가 있다.

이를 종합해보면, 훈련 데이터가 지속적으로 들어올 때 이전 모델을 버리지 않고 계속 들어오는 훈련세트를 훈련에 활용하는 방안이 필요하다.

이 것이 바로 ```Stochastic Gradient Descent (확률적 경사하강법)``` 이다.

```Stochastic``` 은 확률적 이라는 의미가 있다. 즉, 랜덤하게 라는 의미다.

```Gradient```은 기울기(경사)를 말한다.

```Descent```은 하강을 말한다.

즉 종합하면, 랜덤하게 경사를 내려오면서 최적의 대상을 찾는 과정(=학습)을 하는 것이 이 확률적경사하강법 이다.

이 때, 경사하강은 훈련세트에서 랜덤하게 하나의 샘플을 활용하여 경사를 조금씩 내려가며, 이 과정을 전체 훈련세트의 모든 샘플들에 대해 수행한다.

전체 훈련세트에 대해 이 과정을 완료하였음에도 불구하고, 경사를 다 내려오지 못했다면, 훈련세트에 모든 샘플들을 다시 채워 넣는다. 이렇게 전체 샘플에 대해 한번씩 경사를 내려오는 학습의 단위를 ```에포크(epoch)```라고 한다.

실제로는 한 번에 하나의 샘플이 아닌, 무작위의 몇 개의 샘플을 꺼내 경사를 내려오는데, 이 방식을 ```미니배치 경사 하강법(minibatch gradient descent)``` 라고 한다.

# 손실 함수 (loss function)

손실 함수는 머신러닝 알고리즘의 성능을 평가하는 척도로, 손실값이 낮을 수록 더욱 뛰어난 성능이라고 할 수 있다. 즉 이 손실함수의 값을 최소화 하는 것이 훈련의 과정이다.

분류에서 '손실'은 아주 확실하다. 정답을 못 맞히는 것
4개 샘플 중 2개를 맞힌다면 정확도는 0.5이다.
그러나 이 정확도가 손실함수로 적절할까?

경사하강법은 말 그대로 '연속'된 '경사'를 내려가면서 최적의 계수를 찾는 학습방법이다. 즉 어느 지점에 대해 미분 가능한 연속적인 손실함수를 사용해야 경사를 계산할 수 있고 이를 바탕으로 경사를따라 내려갈 수 있기 때문에, 손실함수로는 연속 함수를 사용해야 한다.